In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
import csv
import pandas as pd
import io
import os
import cv2
from pandas.errors import EmptyDataError
import matplotlib.pyplot as plt

drive.mount('/drive')

dataset = pd.read_csv('/drive/My Drive/Diabatic_Retinopathy/train.csv',error_bad_lines=False)
image_folder_path = "/drive/My Drive/Diabatic_Retinopathy/train_images"
processed_image_folder_path = "/drive/My Drive/Diabatic_Retinopathy_Preprocessed_Images"

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

def applying_gaussian_filter(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (300, 300))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

def process_images(row, index):
  image_path = os.path.join(image_folder_path, row['id_code']+'.png')
  image = applying_gaussian_filter(image_path,40)
  print(row['id_code'], row['diagnosis'])
  # plt.imshow(image,cmap='gray')
  # plt.show()

  image_write_path = os.path.join(processed_image_folder_path, row['id_code']+'.png')
  cv2.imwrite(image_write_path, image)

for index, row in dataset.iterrows():
    process_images(row, index)